# Text Analytics - Sentiment

## Description  
This use case walks through a sentiment analysis of text strings use case. Participants will access a Jupyter notebook and run through a series of commands that will load data, create a sentiment model, score the model, and then process the results in SAS Studio. To complete the task, hackers will be asked a series of questions about the resulting data set.

## Objectives  
- Create a data set and apply the sentiment Text Anlytics Models through REST requests using the sentimentAnalysis.applySent CAS action.  
- Analyze the results in SAS Studio

## Import modules, variable assignment
The first step of the process is to import the required packages and assign variable values.

## Create CAS connection to start making calls

In [ ]:
import requests
import json
import os

## Variables to assign

sasserver = "<enter SAS Viya server URL here>"
text = " Today it rained; we didn't go to school\r\n456, Our team won the game\r\n789, The funeral was a sad event\r\n012, The quick brown fox jumped over the lazy dog\r\n345, What a long strange trip it's been\r\n678, The telephone was rang and I handed it to Liz. She said \"This isn't who it would be If it wasn't who it is\"\r\n901, She was having a no good terrible very bad day\r\n234, If I could be the sun I'd radiate like Africa and Smile upon the world Intergalactic love laughter"
access_token = os.environ['ACCESS_TOKEN'] # Get from the authentication project

In [30]:
import swat
conn = swat.CAS(os.environ['CAS_CONTROLLER'], 5570, password=access_token)
conn

CAS('controller.sas-cas-server-default.viya', 5570, 'joe.furbee@sas.com', protocol='cas', name='py-session-3', session='3d5e9ef9-1de3-6b48-b32b-92b81e9e29d6')

In [31]:
# Get session id and print it out
sessionId = conn.sessionid().session
sessionId

'3d5e9ef9-1de3-6b48-b32b-92b81e9e29d6'

## Load data from the form to be scored

In [ ]:
def csv_load(sessionId, access_token, sasserver, data):

   url = sasserver + "/cas-shared-default-http/cas/sessions/"+sessionId+"/actions/upload"
   payload = "UID,Text\r\n"+data

   headers = {
      'Accept': 'application/json',
      'Content-Type': 'binary/octet-stream',
      'JSON-Parameters': '{"casout":{"caslib":"casuser","name":"tableToScore","replace":true},"importOptions":{"fileType":"csv"}}',
      'Authorization': 'Bearer '+ access_token
   }


   try:
      response = requests.request("PUT", url, headers=headers, data = payload, verify=False)
      return str(response)
   except requests.exceptions.RequestException as e:
      return e

In [ ]:
# Upload Data
unique_id = 123
parsedData = str(unique_id)+","+text
print(parsedData)

# Load CSV
load_data = csv_load(sessionId, access_token, sasserver, parsedData)
print(load_data)

## Run Sentiment Model

In [ ]:
def callSentimentModel(sessionId, access_token, sasserver):
   headers = {
      "Accept": "application/json",
      "Content-Type": "application/json",
      "Authorization": "Bearer " + access_token }

   url = sasserver + '/cas-shared-default-http/cas/sessions/'+sessionId+'/actions/sentimentAnalysis.applySent'

   payload = "{\"table\":{\"caslib\":\"casuser\", \"name\":\"tableToScore\"},\r\n\"docId\":\"uid\",\r\n\"text\":\"text\",\r\n\"language\":\"ENGLISH\",\r\n\"casOut\":{\"caslib\":\"casuser\", \"name\":\"sentimentAnalysis\", \"promote\":true}}"

   try:
      response = requests.request("POST", url, headers=headers, data=payload, verify=False).json()
      return(response)
   except requests.exceptions.RequestException as e:
      return e

In [ ]:
# Run Sentiment Model
sentiment_output = callSentimentModel(sessionId, access_token, sasserver)
sentiment_output_format = json.dumps(sentiment_output, indent=2)
print(sentiment_output_format) # optional print statement to view output

## Get the data using the Fetch action

In [ ]:
def getData(sasserver, sessionId, access_token, tblName):
   url = sasserver + "/cas-shared-default-http/cas/sessions/" + sessionId + "/actions/table.fetch"

   payload = "{ \"table\": {\"caslib\":\"casuser\", \"name\":\"" + tblName + "\"} }"

   headers = {
      'Authorization': "Bearer " + access_token,
      'Content-Type': 'application/json'
   }

   try:
      response = requests.request("POST", url, headers=headers, data = payload, verify=False).json()
      return(response)
   except requests.exceptions.RequestException as e:
      return e    

In [ ]:
# Get Sentiment data
sentiment_data = getData(sasserver, sessionId, access_token, "sentimentAnalysis")
sentiment_format = json.dumps(sentiment_data, indent=2)
print(sentiment_format)

with open("sentiment_data.json", "w") as file:
    file.write(str(sentiment_format))

## Drop table at end of use case

In [ ]:
def dropTable(sasserver, sessionId, access_token):
   url = sasserver + "/cas-shared-default-http/cas/sessions/" + sessionId + "/actions/table.dropTable"

   payload = "{\"caslib\": \"casuser\", \"name\": \"sentimentanalysis\"}"

   headers = {
       'Authorization': 'Bearer '+ access_token,
       'Content-Type': 'application/json'
   }

   try:
      response = requests.request("POST", url, headers=headers, data=payload, verify=False).json
      return(response)
   except requests.exceptions.RequestException as e:
      return e

In [ ]:
# Drop table
drop_table = dropTable(sasserver, sessionId, access_token)

## End session

In [ ]:
def endSession(sasserver, sessionId, access_token):
   url = sasserver + "/cas-shared-default-http/cas/sessions/" + sessionId + "/actions"

   headers = {
      'Authorization': "Bearer " + access_token,
      'Content-Type': 'application/json'
   }

   try:
      response = requests.request("DELETE", url, headers=headers, verify=False).json
      return(response)
   except requests.exceptions.RequestException as e:
      return e

In [ ]:
# End session
endSession= endSession(sasserver, sessionId, access_token)
print(endSession)